# setup

In [1]:
# load openai_api_key from .env file
from dotenv import load_dotenv
import os

load_dotenv()
openai_api_key = os.environ.get('OPENAI_API_KEY')

model="gpt-4"


In [2]:
# import of langchain
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema import StrOutputParser

# copy-paste a media release as 'text_input_medienmitteilung'

In [3]:
text_input_medienmitteilung = """

    # Medienmitteilung

    ## Ouverture de la consultation relative à la révision partielle de l’ordonnance régissant la taxe sur la valeur ajoutée
    
    Berne, 25.10.2023 - Lors de sa séance du 25 octobre 2023, le Conseil fédéral a ouvert la procédure de consultation relative à la révision partielle de l’ordonnance régissant la taxe sur la valeur ajoutée (OTVA). Cette révision porte notamment sur des dispositions d’exécution découlant de la révision partielle de la loi sur la TVA (LTVA), des simplifications concernant les décomptes au moyen des taux de la dette fiscale nette et des taux forfaitaires et une extension des démarches devant être effectuées en ligne.

    La révision partielle de la LTVA adoptée par le Parlement en juin a permis de mettre en œuvre plusieurs motions transmises par les Chambres fédérales. Les modifications apportées à la loi nécessitent maintenant aussi des changements dans l’ordonnance, par exemple en ce qui concerne l’imposition des plateformes en ligne et la nouvelle possibilité d’établir un décompte annuel.

    Le Conseil fédéral profite de cette révision pour mettre à jour et préciser l’OTVA dans d’autres domaines également. Dans son projet, il remanie notamment en profondeur les méthodes des taux de la dette fiscale nette et des taux forfaitaires pour accroître l’exactitude dans la détermination de la dette fiscale et simplifier ces méthodes de décompte de la TVA, pour les petites et moyennes entreprises entre autres. À titre de mesure de lutte contre la fraude, le projet prévoit par ailleurs l’obligation d’appliquer la procédure de déclaration en cas de paiement en espèces de 15 000 francs ou plus pour des prestations imposables entre personnes assujetties.

    Enfin, le projet prescrit l’utilisation du portail électronique pour le dépôt des demandes et les annonces relatives aux méthodes, toutes facultatives, des taux de la dette fiscale nette et des taux forfaitaires, du décompte annuel et de l’imposition de groupe.

    La révision partielle de l’OTVA devrait entrer en vigueur le 1er janvier 2025. Cette date est aussi celle visée pour l’introduction du décompte annuel, qui nécessite d’importants changements informatiques.

    La consultation prendra fin le 8 février 2024.

    Adresse pour l'envoi de questions
    Administration fédérale des contributions AFC
"""

# run this ai-machine

In [4]:
# result_facts: Analyze the following text and identify all important facts
prompt = PromptTemplate.from_template("""
Analyze the following text and 
    identify all important facts.  
    The text is from news.admin.ch and is an 
    official media release: \n {medienmitteilung}\n\n
    a fact analysis in the same language as
    the media release:
""")
runnable = prompt | ChatOpenAI(model=model) | StrOutputParser()
result_facts = runnable.invoke({"medienmitteilung": text_input_medienmitteilung})
print(result_facts)

1. Le Conseil fédéral a ouvert la procédure de consultation relative à la révision partielle de l’ordonnance régissant la taxe sur la valeur ajoutée (OTVA) lors de sa séance du 25 octobre 2023.
2. La révision porte sur des dispositions d’exécution découlant de la révision partielle de la loi sur la TVA (LTVA), des simplifications concernant les décomptes au moyen des taux de la dette fiscale nette et des taux forfaitaires et une extension des démarches devant être effectuées en ligne.
3. La révision partielle de la LTVA adoptée par le Parlement en juin a permis de mettre en œuvre plusieurs motions transmises par les Chambres fédérales.
4. Les modifications apportées à la loi nécessitent maintenant aussi des changements dans l’ordonnance, notamment en ce qui concerne l’imposition des plateformes en ligne et la nouvelle possibilité d’établir un décompte annuel.
5. Le Conseil fédéral remanie les méthodes des taux de la dette fiscale nette et des taux forfaitaires pour accroître l’exactitu

In [5]:
# result_top10keywords: Identify 10 keywords from the facts
prompt = PromptTemplate.from_template("""
    Identify 10 keywords that are likely to rank well 
    on Google (Switzerland). 
    The keywords are in the same language as the 
    following text and start with a #.

    Text: 
    {facts}
    
    Examples: #FederalCouncil #Geneva #Nyon #Motorway #A1 #Traffic #Trafficflow #Motorway     
    Your keywords incl. #:
""")
runnable = prompt | ChatOpenAI(model=model) | StrOutputParser()
result_top10keywords = runnable.invoke({"facts": result_facts})
print(result_top10keywords)

#ConseilFédéral #RévisionPartielle #OTVA #LTVA #TVA #PlateformesEnLigne #DécompteAnnuel #DetteFiscale #PortailÉlectronique #AdministrationFédéraleDesContributions


In [6]:
# result_top3keywords: Analyze the following keywords and text. 
prompt = PromptTemplate.from_template("""
    Analyze the following keywords and text. 
    Identify exactly 3 keywords that can rank 
    well on Google (Switzerland). 

    Examples: #FederalCouncil #ChuckNorris #Thailand  (on one line)
   
    ===============
    Keywords: 
    {top10keywords}

    Text: 
    {medienmitteilung}
    ===============
    Your top 3 keywords in the same language as the text/keywords:
""")
runnable = prompt | ChatOpenAI(model=model) | StrOutputParser()
result_top3keywords = runnable.invoke({"medienmitteilung": text_input_medienmitteilung, "top10keywords": result_top10keywords})
print(result_top3keywords)

#ConseilFédéral #RévisionPartielle #AdministrationFédéraleDesContributions


In [7]:
# result_tweets: write 3 matching tweets for a specific media release, facts, keywords.
prompt = PromptTemplate.from_template("""
    You are a neutral media spokesperson for the Federal Council 
    and write 3 matching tweets for a specific media release.
    
    Your tweets always contain:
    1. the 2 most important facts
    2. the 3 most important keywords
    3. 4 matching emojis.
    4. the tweet ends with the string [url].

    Rules:
    You answer in the same language as the media release.

    Sources: 
    ===========
    Facts: {facts}
    Top-Keywords: {top3keywords}
    Media release: {medienmitteilung}
    ==============
    Your 3 different tweet suggestions:
""")
runnable = prompt | ChatOpenAI(model=model) | StrOutputParser()
result_tweets = runnable.invoke({"medienmitteilung": text_input_medienmitteilung, "top3keywords": result_top3keywords, "facts": result_facts})
print(result_tweets)

1. "📢📜 Le #ConseilFédéral a ouvert la procédure de consultation relative à la #RévisionPartielle de l’OTVA. 🎯 Principaux changements : simplifications des décomptes et extension des démarches en ligne. Date d'entrée en vigueur : 1er janvier 2025. [url] 📅💻"

2. "🔍⚖️ La révision partielle de l'OTVA vise à augmenter l'exactitude dans la détermination de la dette fiscale et à simplifier les méthodes de décompte de la TVA. 🚀 Intégration d'un décompte annuel à partir de 2025. #ConseilFédéral #RévisionPartielle [url] 📈📊"

3. "👥🔐 La #RévisionPartielle de l'OTVA par le #ConseilFédéral impose l'utilisation du portail électronique pour le dépôt des demandes et les annonces. ✍️ La consultation prendra fin le 8 février 2024. Questions? Adressez-vous à l'#AdministrationFédéraleDesContributions. [url] 📧📬"


In [8]:
# result_fragen_20min: from 20min.ch/Wagenknecht and will collect 3 critical questions. 
prompt = PromptTemplate.from_template("""
    You are a reporter (male) for 20min.ch and 
    will collect 10 critical questions. 
    At the end we need the 3 most critical 
    questions.

    You think like Sahra Wagenknecht. 

    It is important that the questions 
    directly to the press release.
    
    Important:
    The questions are critical but not offensive. 
    and refer to the media release, the facts and a top keyword.

    Rules:
    Answer in the same language as the media release.

    Sources: 
    ===========
    Facts: {facts}
    Top-Keywords: {top3keywords}
    Media release: {medienmitteilung}
    ==============
    Your top-3 moust critical questions:
""")
runnable = prompt | ChatOpenAI(model=model) | StrOutputParser()
result_fragen_20min = runnable.invoke({"medienmitteilung": text_input_medienmitteilung, "top3keywords": result_top3keywords, "facts": result_facts})
print(result_fragen_20min)

1. Comment le Conseil fédéral justifie-t-il l'extension des démarches à effectuer en ligne, notamment en termes de garanties de la sécurité des données et de la possibilité pour tous les citoyens d'y accéder facilement ? 
2. Pouvez-vous détailler davantage les modifications prévues pour les taux de la dette fiscale nette et les taux forfaitaires ? Quels seront les impacts concrets pour les petites et moyennes entreprises ? 
3. Comment le Conseil fédéral compte-t-il lutter efficacement contre la fraude fiscale, notamment en ce qui concerne l'obligation de déclaration pour les paiements en espèces de 15 000 francs ou plus ? Cette mesure ne risque-t-elle pas de pénaliser les entreprises légitimes plutôt que de cibler les fraudeurs ?


In [9]:
# result_fragen_NZZ: from NZZ/Donald Duck and will collect 3 critical questions. 
prompt = PromptTemplate.from_template("""
You are a reporter (female) for the NZZ and will 
    collect 10 critical questions. 
    At the end we need the 3 most critical 
    questions.

    You think like Donald Duck.

    It is important that the questions 
    relate directly to the press release.
    
    This is important:
    The questions are critical, but not insulting. 
    And refer to the press release, the facts and a keyword.

    Rules:
    Answer in the same language as in the press release.

    Sources: 
    ===========
    Facts {facts}
    Keywords: {top3keywords}
    Media Release: {medienmitteilung}
    ==============
    Your top 3 most critical questions:
""")
runnable = prompt | ChatOpenAI(model=model) | StrOutputParser()
result_fragen_nzz = runnable.invoke({"medienmitteilung": text_input_medienmitteilung, "top3keywords": result_top3keywords, "facts": result_facts})
print(result_fragen_nzz)

1. Quels sont les défis spécifiques que le Conseil fédéral envisage dans la mise en œuvre de l'obligation de paiement en espèces de 15 000 francs ou plus pour des prestations imposables entre personnes assujetties, et comment prévoit-il de les surmonter ?
2. Quels seront les impacts anticipés de l’imposition des plateformes en ligne et de l'introduction du décompte annuel sur les petites et moyennes entreprises ? 
3. Comment l'Administration fédérale des contributions AFC s'assurera-t-elle que les changements informatiques nécessaires à l'introduction du décompte annuel seront effectués sans problème et à temps pour la date d'entrée en vigueur prévue du 1er janvier 2025 ? 

10 questions critiques :
1. Quels sont les principaux avantages que le Conseil fédéral espère obtenir avec cette révision partielle de l'OTVA ? 
2. Comment les simplifications concernant les décomptes au moyen des taux de la dette fiscale nette et des taux forfaitaires seront-elles mises en œuvre ?
3. Quels sont les

In [10]:
# result_zusammenfassung_kind: summary for a 10 year old child 
prompt = PromptTemplate.from_template("""
Create a summary of the following text 
    for a 10 year old child of a media release.

    Important:
    Answer in the same language as the media release.

    Sources: 
    ===========
    Facts: {facts}
    Media release: {medienmitteilung}
    ==============
    Your child-friendly summary:
""")
runnable = prompt | ChatOpenAI(model=model) | StrOutputParser()
result_zusammenfassung_kind =  runnable.invoke({"medienmitteilung": text_input_medienmitteilung, "facts": result_facts})
print(result_zusammenfassung_kind)

Le gouvernement suisse envisage de modifier certaines règles concernant la taxe sur la valeur ajoutée (TVA), qui est une taxe que tu paies lorsque tu achètes quelque chose. Ils veulent simplifier le système pour les petites et moyennes entreprises et rendre certaines choses plus précises. Ils veulent aussi que plus de choses soient faites en ligne. Par exemple, si quelqu'un paie plus de 15 000 francs en espèces pour quelque chose, ils devront le déclarer en ligne. Ces changements devraient commencer à partir du 1er janvier 2025. Les gens peuvent donner leur avis sur ces modifications jusqu'au 8 février 2024.


In [11]:
# result_dalle3prompt: DALL-E prompt : black and white comic book panel in style of asterix & obelix in mid-21th 
# copy-paste the result_dalle3prompt into a good image generator like Midjourney or DallE-3 
prompt = PromptTemplate.from_template("""
Create a prompt to generate 
    with DALL-E a matching image.

    Important Roules:
    =================
    1) Your DALL-E Prompt must be in english.
    2) It must include the following texts:
        A)"create a black and white comic book panel"
        B)"comic illustration reminiscent of European adventure comics of the mid-21th century"
        C)"in a informative way." 
        D) [SCENE]
        E) The scene plays in switzerland.
    3) "Include a dramatic sound effect text saying &quot;{top3keywords}&quot;
    =================

    Contextinformation for the [SCENE]: {zusammenfassung_kind}
    ==============
    Generated the [SCENE] and following the rules 1-3!
    ==============
    Your DALL-E Prompt follows rules 1-3:
""")
runnable = prompt | ChatOpenAI(model=model) | StrOutputParser()
result_dalle3prompt =  runnable.invoke({"zusammenfassung_kind": result_zusammenfassung_kind, "top3keywords": result_top3keywords, "facts": result_facts})
print(result_dalle3prompt)

"Create a black and white comic book panel featuring a scene set in Switzerland, where the Swiss government is considering modifying some rules regarding Value Added Tax (VAT). The comic illustration should be reminiscent of European adventure comics of the mid-21th century, and it should depict the government's plan to simplify the system for small and medium businesses, make certain things more precise, and encourage more things to be done online. For instance, if someone pays more than 15,000 francs in cash for something, they will have to declare it online. These changes are expected to begin from January 1, 2025, and people can give their opinions on these modifications until February 8, 2024. The scene should be presented in an informative way. Include a dramatic sound effect text saying "#ConseilFédéral #RévisionPartielle #AdministrationFédéraleDesContributions"."


In [12]:
# link to a DELL-E 2 image (Midjourney or DallE-3 are much better!)
from langchain.utilities.dalle_image_generator import DallEAPIWrapper
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.9)
prompt = PromptTemplate(
    input_variables=["image_desc"],
    template="{image_desc}",
)
chain = LLMChain(llm=llm, prompt=prompt)
image_url = DallEAPIWrapper().run(chain.run(result_dalle3prompt))
image_url

'https://oaidalleapiprodscus.blob.core.windows.net/private/org-RrVvFp3h2smAtr8Vjq0g3AKy/user-O767lIr6xCB4R6abFFZNETBT/img-LrPIum6BuVoIkdxGt0P7B8qo.png?st=2023-10-29T17%3A51%3A18Z&se=2023-10-29T19%3A51%3A18Z&sp=r&sv=2021-08-06&sr=b&rscd=inline&rsct=image/png&skoid=6aaadede-4fb3-4698-a8f6-684d7786b067&sktid=a48cca56-e6da-484e-a814-9c849652bcb3&skt=2023-10-28T23%3A32%3A15Z&ske=2023-10-29T23%3A32%3A15Z&sks=b&skv=2021-08-06&sig=9LmQY9cljVjBRBPrOPjE2g/sE%2BkN66tNtN6ZuH3SBJA%3D'